# Getting City Elevation Data
## Steps
1. Determine elevation for each tree via longitude and latitude
2. Save new field, `elevation`, into tree files

In [1]:
import requests

def get_elevation(lat, long):
    query = f"https://api.open-elevation.com/api/v1/lookup?locations={lat},{long}"
    response = requests.get(query).json()
    return response['results'][0]['elevation']

# Example usage:
elevation = get_elevation(40.7142700, -74.0059700)  # Coordinates for New York City
print(f"Elevation: {elevation} meters")

Elevation: 51.0 meters


In [6]:
# Now, let's loop over each file
import os
FILEPATH = "/Users/owen/Development/tmp/bio-data-analysis/data/trees/" # Update with your project path
csv_files = os.listdir(FILEPATH)
csv_files

['Milwaukee_Final_2022-06-18.csv',
 'Anaheim_Final_2022-06-18.csv',
 'Irvine_Final_2022-06-18.csv',
 'RanchoCucamonga_Final_2022-06-18.csv',
 'Arlington_Final_2022-06-18.csv',
 'Rochester_Final_2022-06-18.csv',
 'Miami_Final_2022-06-18.csv',
 'Austin_Final_2022-06-18.csv',
 'Stockton_Final_2022-06-18.csv',
 'Seattle_Final_2022-06-18.csv',
 'LosAngeles_Final_2022-06-18.csv',
 'AuroraCO_Final_2022-06-18.csv',
 'NewYork_Final_2022-06-18.csv',
 'Oakland_Final_2022-06-18.csv',
 'SanFrancisco_Final_2022-06-18.csv',
 'Boston_Final_2022-06-18.csv',
 'Ontario_Final_2022-06-18.csv',
 'GrandRapids_Final_2022-06-18.csv',
 'Baltimore_Final_2022-06-18.csv',
 'Albuquerque_Final_2022-06-18.csv',
 'Greensboro_Final_2022-06-18.csv',
 'OklahomaCity_Final_2022-06-18.csv',
 'HuntingtonBeach_Final_2022-06-18.csv',
 'Fresno_Final_2022-06-18.csv',
 'Orlando_Final_2022-06-18.csv',
 'Nashville_Final_2022-06-18.csv',
 'StLouis_Final_2022-06-18.csv',
 'Tampa_Final_2022-06-18.csv',
 'LasVegas_Final_2022-06-18.csv'

In [7]:
def separate_caps(text):
  """Inserts spaces between uppercase letters in a string, unless they're consecutive.

  Args:
      text: The string to separate.

  Returns:
      A new string with spaces inserted between uppercase letters.
  """
  result = []
  prev_was_upper = False
  for char in text:
    if char.isupper():
      if not prev_was_upper:
        result.append(' ')
      result.append(char)
      prev_was_upper = True
    else:
      result.append(char)
      prev_was_upper = False
  return ''.join(result)

# Test cases
print(separate_caps("AuroraCO"))  # Output: Aurora CO
print(separate_caps("GrandRapids"))  # Output: Grand Rapids
print(separate_caps("USA"))  # Output: USA
print(separate_caps("TheUSA"))  # Output: USA

 Aurora CO
 Grand Rapids
 USA
 The USA


In [8]:
!pip install geopy joblib
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderServiceError
from joblib import Memory
location = './cachedir'
memory = Memory(location, verbose=0)

@memory.cache
def get_city_coordinates(city_query):
    # At each capitalization, add a space
    try:
        geolocator = Nominatim(user_agent="city_coordinates_for_bio_lab")
        location = geolocator.geocode(separate_caps(city_query), timeout=10)
        
        if location:
            return (location.latitude, location.longitude)
        else:
            return None
    except (GeocoderTimedOut, GeocoderServiceError) as e:
        print(f"Error: {e}")
        return None

# Example usage
city_query = "AuroraCO"
coordinates = get_city_coordinates(city_query)

if coordinates:
    print(f"The coordinates of {city_query} are: {coordinates}")
else:
    print(f"Could not find coordinates for {city_query}")

The coordinates of AuroraCO are: (43.6963371, -98.5722461)


In [12]:
import csv


for file in csv_files:
  name = file.split("_")[0]
  path = os.path.join(FILEPATH, file)
  # Check if 'elevation' already exists in the header
  already_elevation = False
  with open(path, "r") as f:
    for line in f:
        if "elevation" in line.lower(): # Case-insensitive search
            already_elevation = True
            break
  if already_elevation:
      print(file, "Already contains elevation data. Skipping now.")
  coors = get_city_coordinates(name)
  this_elevation = get_elevation(coors[0], coors[1])
  # We calculate the elevation for the city
  print("Going over file: ", name)
  path = os.path.join(FILEPATH, file)
  if name == "RanchoCucamonga":
      this_elevation = 368 # Doesn't work with OpenStreetMap for whatever reason
        

  with open(path, "r") as f:
      reader = csv.reader(f, delimiter=",")
      lines = list(reader)

      # Add the new 'elevation' field to the header
      lines[0].append('elevation')

      # Add the default elevation value to each row
      for row in lines[1:]:
        row.append(this_elevation)

      # Write the updated content back to the file
  with open(path, "w", newline="") as f:
        writer = csv.writer(f, delimiter=",")
        writer.writerows(lines)


Going over file:  Milwaukee
Going over file:  Anaheim
Going over file:  Irvine
Going over file:  RanchoCucamonga
Going over file:  Arlington
Going over file:  Rochester
Going over file:  Miami
Going over file:  Austin
Going over file:  Stockton
Going over file:  Seattle
Going over file:  LosAngeles
Going over file:  AuroraCO
Going over file:  NewYork
Going over file:  Oakland
Going over file:  SanFrancisco
Going over file:  Boston
Going over file:  Ontario
Going over file:  GrandRapids
Going over file:  Baltimore
Going over file:  Albuquerque
Going over file:  Greensboro
Going over file:  OklahomaCity
Going over file:  HuntingtonBeach
Going over file:  Fresno
Going over file:  Orlando
Going over file:  Nashville
Going over file:  StLouis
Going over file:  Tampa
Going over file:  LasVegas
Going over file:  Phoenix
Going over file:  Minneapolis
Going over file:  Indianapolis
Going over file:  NewOrleans
Going over file:  Madison
Going over file:  Columbus
Going over file:  Knoxville
Goin